In [4]:
import numpy as np

Написать программы (2 шт.), реализующие явную и неявную схемы
для уравнения

$$
\begin{array}{l}
u_t + (f(u))_x = 0 \\
u|_{t=0} = u_0(x) \\
x \in [a,b]
\end{array}
$$

Входные данные (которые могут быть изменены по указанию преподавателя):
    
    Нелинейность f (u) (для отладки можно взять f (u) = u^2);
    u0(x) (для отладки можно взять u0(x) = th(−x));
    Начало и конец отрезка и его длина;
    Краевые условия;
    Время окончания вычислений T , t ∈ [0, T ].



# Явная схема

In [5]:
def u_1_m(u, m, tau, h, F):
    return u[m] - (tau/h) * (F(u[m+1]) - F(u[m]))

def heat_expl(init, bound1, bound2, F, N, alpha, beta, tau, T):
    
    
    # YOUR CODE HERE
    w_x, h = np.linspace(alpha, beta, N+1, retstep=True)
    N_t = int(T / tau)
    u_0 = np.array([init(x) for x in w_x], dtype=np.float64)
    u_end = np.zeros(u_0.shape)
    
    for t0 in range(1, N_t+1):
        u_0 = u_end.copy()
        for m in range(1, N):
            u1m = u_1_m(u_0, m, tau, h, F)
            u1mm1 = u_1_m(u_0, m-1, tau, h, F)
            u_end[m] = 0.5 * (u_0[m] + u1m - (tau/h) * (F(u1m) - F(u1mm1)))
        u_end[0] = bound1(tau * t0)
        u_end[-1] = bound2(tau * t0)
    
    return u_end

In [6]:
def F(u):
    return u

In [7]:
from numpy.testing import assert_allclose

u_0 = heat_expl(lambda x: 0., lambda x: 1., lambda x: 1., F, 10,
                     alpha=0., beta=1., tau=0.005, T=2)

# Неявная схема

In [ ]:
from scipy.linalg import solve_banded

def heat_impl(init, bound1, bound2, alpha, lx, h, tau, tol=1e-3):

    
    w_x = np.arange(0, lx+h, h)[:int(lx/h)+1]
    t_0 = 0.
    t_end = t_0 + tau
    u_0 = np.array([init(x) for x in w_x], dtype=np.float64)
    N = u_0.shape[0]
    
    A = np.zeros((N-2,N-2), dtype=np.float64)
    b = np.zeros(N-2, dtype=np.float64)
    u_end = np.zeros(N, dtype=np.float64)
    u_end[0] = bound1(t_end)
    u_end[-1] = bound2(t_end)
    
    A[0][:2] = (2 * alpha * tau / np.power(h, 2) + 1), -(alpha * tau / np.power(h, 2))
    b[0] = u_0[1] + (alpha * tau / np.power(h, 2)) * u_end[0]
    for i in range(1,N-3):
        A[i][i-1], A[i][i], A[i][i+1] = -(alpha * tau / np.power(h, 2)), (2 * alpha * tau / np.power(h, 2) + 1), -(alpha * tau / np.power(h, 2))
        b[i] = u_0[i+1]
    A[-1][-2:] = -(alpha * tau / np.power(h, 2)), (2 * alpha * tau / np.power(h, 2) + 1)
    b[-1] = u_0[-2] + (alpha * tau / np.power(h, 2)) * u_end[-1]
    
    ab = np.zeros((3, A.shape[0]))
    ab[0,1:] = np.diag(A,k=1)
    ab[1] = np.diag(A)
    ab[2,:-1] = np.diag(A, k=-1)
    u_end[1:-1] = solve_banded((1,1),ab,b) 
    
    while np.linalg.norm(u_end - u_0) >= tol:
        u_0 = u_end.copy()
        t_end += tau
        
        u_end = np.zeros(N, dtype=np.float64)
        u_end[0] = bound1(t_end)
        u_end[-1] = bound2(t_end)
        
        b[0] = u_0[1] + (alpha * tau / np.power(h, 2)) * u_end[0]
        for i in range(1,N-3):
            b[i] = u_0[i+1]
        b[-1] = u_0[-2] + (alpha * tau / np.power(h, 2)) * u_end[-1]
        
        u_end[1:-1] = solve_banded((1,1),ab,b)   
        
    u_0 = u_end.copy()
    t_end += tau
    u_end = np.zeros(N, dtype=np.float64)
    u_end[0] = bound1(t_end)
    u_end[-1] = bound2(t_end)
    b[0] = u_0[1] + (alpha * tau / np.power(h, 2)) * u_end[0]
    for i in range(1,N-3):
        b[i] = u_0[i+1]
    b[-1] = u_0[-2] + (alpha * tau / np.power(h, 2)) * u_end[-1]
    u_end[1:-1] = solve_banded((1,1),ab,b) 
    
    return t_end, u_end